# Краткая шпаргалка по gensim

### Что в ней вообще есть

1. Word2Vec, FastText, и Doc2Vec
2. Тематическое моделирование (Topic Modeling). Поддержка LDA (Latent Dirichlet Allocation) и LSI (Latent Semantic Indexing) для извлечения тематической структуры текстов
3. Поддержка LDA (Latent Dirichlet Allocation) и LSI (Latent Semantic Indexing) для извлечения тематической структуры текстов

пусть есть очищенный текст **war_and_peace_cleaned.txt** - без знаков препинания, все в нижнем регистре, предложения - по строкам

In [79]:
from gensim.models import FastText
from gensim.models.word2vec import PathLineSentences


In [80]:
!sed -n 1,4p war_and_peace_cleaned.txt

well prince so genoa and lucca are now just family estates of the buonapartes
but i warn you if you don't tell me that this means war if you still try to defend the infamies and horrors perpetrated by that antichrist i really believe he is antichrist i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself
but how do you do
i see i have frightened you sit down and tell me all the news


In [81]:
wap_cleaned_file_path = 'data/war_and_peace_cleaned.txt'

In [82]:
# забирает из файла строки и бьет их на слова, объединяя в массив
wap_sentences = PathLineSentences(wap_cleaned_file_path)

In [83]:
for sentence in wap_sentences:
    print(sentence)
    break

['well', 'prince', 'so', 'genoa', 'and', 'lucca', 'are', 'now', 'just', 'family', 'estates', 'of', 'the', 'buonapartes']


## FastText

In [84]:
# создает эмбеддинги по методу FastText
model = FastText(
        wap_sentences,
        vector_size=100, # размерность эмбеддинга
        window=5,        # размер окна
        alpha=3e-2,      # лёнин рейт
        sg=1,            # skip-gram
        epochs=5,        # число эпох
        seed=42,
    )

model.save("data/wap_fasttext_model.bin")

#### в  model.wv теперь все самое интересное и там много всего

In [85]:
# 10 самых близких слов
print(*model.wv.most_similar('peace', topn=10), sep='\n')

('secrecy', 0.8486680388450623)
('decisive', 0.8451395034790039)
('freedom', 0.8440701961517334)
('religion', 0.842451274394989)
('problem', 0.8415459394454956)
('insufficient', 0.8414140939712524)
('decision', 0.8381208777427673)
('fulfill', 0.837116539478302)
('source', 0.836845338344574)
('conquer', 0.8368210196495056)


In [86]:
# токены и их индексы в массиве
print(model.wv.key_to_index['peace'])
print(model.wv.index_to_key[700])
print('light' in model.wv.key_to_index)

578
calm
True


In [87]:
# слова наиболее близкие к слову
print(*model.wv.most_similar('light', topn=10), sep='\n')

('lightblue', 0.9081708788871765)
('moonlight', 0.9040265083312988)
('bright', 0.8982992768287659)
('plight', 0.885283887386322)
('slight', 0.8809515833854675)
('sky', 0.871822714805603)
('twilight', 0.8711280822753906)
('daylight', 0.8605808615684509)
('flight', 0.8605671525001526)
('dark', 0.8552883267402649)


In [88]:
# параметры можно выдергивать из модели
print(model.alpha, model.sg)
model_vocabulary = set(model.wv.key_to_index.keys())

0.03 1


#### дообучение уже готовых эмбеддингов на другом корпусе

In [89]:
finetuned_model = FastText.load("data/wap_fasttext_model.bin")

# хотим дообучить эмбеддинги на новом корпусе - не трогаем то что уже обучено
# но добавляем новые слова в словарь 
quora_processed_file_path = 'data/quora_processed.txt'
quora_sentences = PathLineSentences(quora_processed_file_path)

# число строу в файле
quora_sentences_count = 50000 #537272

# update=True означает добавлять новые слова
finetuned_model.build_vocab(quora_sentences, update=True)

finetuned_model.alpha = finetuned_model.min_alpha * .9 # делаем поменьше

# Remember to correctly pass the parameters
finetuned_model.train(
    quora_sentences, 
    total_examples=quora_sentences_count, 
    epochs=finetuned_model.epochs - 1, # эпох тоже можно сделать поменьше, а можно и не делать
)

finetuned_model.save("data/wap_quora_fasttext_model.bin")

In [90]:
finetuned_model_vocabulary = set(finetuned_model.wv.key_to_index.keys())
new_words = finetuned_model_vocabulary - model_vocabulary

print("Number of new words added:", len(new_words))
print("New words:", list(new_words)[10:20])

Number of new words added: 23840
New words: ['termination', 'kick', 'tungsten', 'pie', 'consolidated', 'subcultures', 'nazis', 'descriptive', 'paella', 'pitcher']


## Тематическое моделирование

In [91]:
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

texts = [["мир", "труд", "май"], ["май", "работа", "отдых"]]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

lda = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=10)
topics = lda.print_topics(num_words=3)

In [92]:
topics

[(0, '0.274*"май" + 0.269*"труд" + 0.269*"мир"'),
 (1, '0.271*"май" + 0.271*"работа" + 0.271*"отдых"')]

In [93]:
lda

## Similarity Queries

In [94]:
texts = [["мир", "труд", "май"], ["май", "работа", "отдых"], ['яблоко','слива','груша','перец','перец']]
dictionary = Dictionary(texts)
print(dictionary[0])
corpus = [dictionary.doc2bow(text) for text in texts]

# по сути индекс слова в словаре и число вхождений
print(corpus[0], corpus[1], corpus[2])

from gensim.similarities import MatrixSimilarity
similarity_index = MatrixSimilarity(corpus)
sims = similarity_index[corpus[0]]
print("\nматрица схожести:",sims)

май
[(0, 1), (1, 1), (2, 1)] [(0, 1), (3, 1), (4, 1)] [(5, 1), (6, 2), (7, 1), (8, 1)]

матрица схожести: [0.99999994 0.3333333  0.        ]


In [95]:
sims

array([0.99999994, 0.3333333 , 0.        ], dtype=float32)

In [97]:
texts = [["мир", "труд", "май"], ["май", "работа", "отдых"], ["мир", "май", "май"]]

from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus)  # Построение TF-IDF модели
corpus_tfidf = [tfidf[doc] for doc in corpus]  # Преобразование в TF-IDF

similarity_index = MatrixSimilarity(corpus_tfidf)  # Матрица схожести
sims = similarity_index[corpus_tfidf[0]]  # Сравнение первого документа со всеми
sims

array([1.        , 0.        , 0.34624156], dtype=float32)